In [7]:
# -*- coding: utf-8 -*-

#---------------------------------------------
import sys 
import qpython.qconnection as qp
from qpython.qconnection import MessageType
import pandas as pd
import numpy as np
import datetime as dt
import tushare as ts
import json
import kdb_csv as kc
#------------------------------------------------
with open('D:/Quant/Local/Strategy/Big_Brain/configs/config_paths.json', 'r') as f: config_paths = json.loads(f.read())
with open('D:/Quant/Local/Strategy/Big_Brain/configs/config_download.json', 'r') as f: config_download = json.loads(f.read())

# with open('configs/config_paths.json', 'r') as f: config_paths = json.loads(f.read())
# with open('configs/config_download.json', 'r') as f: config_download = json.loads(f.read())
    
for i in config_paths: sys.path.append(config_paths[i])
sys.path.append('D:/Quant/Local/Strategy/Big_Brain/Data_Management/Download')

# #---------------------------------------------
# import HDB 
import query_api

In [12]:
class HDB_Core():
    def __init__(self,config_download):
        self.config = config_download
        self.save_use_csv_or_kdb = self.config['use_csv_or_kdb']
    def reset_HDB_check(self):
        self.kdb_initial_download_list = []
        self.csv_initial_download_list = []
        self.kdb_update_list = []
        self.csv_update_list = []
        self.whether_download_initial_KDB = 0
        self.whether_download_initial_CSV = 0
        self.whether_update_KDB = 0
        self.whether_update_CSV = 0
        self.today = pd.to_datetime(dt.date.today())

    def get_last_date_of_file(self,file_type):
        if file_type == 'KDB':
            for i in self.config['files_stored']:
                self.config['use_csv_or_kdb'] = 'kdb'
                data = kc.load(self.config,i)
                self.config['use_csv_or_kdb'] = self.save_use_csv_or_kdb
                data['trade_date'] += 946684800000000000
                data['trade_date'] = pd.to_datetime(data['trade_date'])
                last_KDB_date_check = pd.to_datetime(data['date'].max().date())
                if last_KDB_date_check<= self.today:
                    if (self.config['runD']): print(dt.datetime.now(), '检查: KDB文件已经是最新: '+str(i)+'; HDB最新日期: '+str(last_KDB_date_check)+'; 当前日期: '+str(self.today))
                else:
                    if (self.config['runD']): print(dt.datetime.now(), '检查: KDB文件需要更新: '+str(i)+'; HDB最新日期: '+str(last_KDB_date_check)+'; 当前日期: '+str(self.today))
                    self.whether_update_KDB = 1
                    self.kdb_update_list.append(i)

        if file_type == 'CSV':
            for i in self.config['files_stored']:
                self.config['use_csv_or_kdb'] = 'csv'
                data = kc.load(self.config, i)
                self.config['use_csv_or_kdb'] = self.save_use_csv_or_kdb
                last_KDB_date_check = pd.to_datetime(data['date'].max().date())
                if last_KDB_date_check<= self.today:
                    if (self.config['runD']): print(dt.datetime.now(), '检查: CSV文件已经是最新: '+str(i)+'; HDB最新日期: '+str(last_KDB_date_check)+'; 当前日期: '+str(self.today))
                else:
                    if (self.config['runD']): print(dt.datetime.now(), '检查: CSV文件需要更新: '+str(i)+'; HDB最新日期: '+str(last_KDB_date_check)+'; 当前日期: '+str(self.today))
                    self.whether_update_CSV = 1
                    self.csv_update_list.append(i)

    def check_HDB_update(self):
        if self.config['save_as_kdb']:
            if (self.config['runD']): print(dt.datetime.now(), '检查: 是否存在原始KDB文件')
            dir = os.listdir(self.config['place_save']['kdb_root'] + self.config['place_save']['kdb'])
            for i in self.config['files_stored']:
                if i in dir: pass
                else: self.kdb_initial_download_list.append(i)
            if len(self.kdb_initial_download_list) > 0:
                if (self.config['runD']): print(dt.datetime.now(), '检查: 缺失以下KDB文件: 将下载最新的以下KDB文件: '+str(self.kdb_initial_download_list))
                self.whether_download_initial_KDB = 1
            else:
                if (self.config['runD']): print(dt.datetime.now(), '检查: 不缺少原始KDB文件,开始检查文件是否最新')

        else:
            if (self.config['runD']): print(dt.datetime.now(), '检查: 跳过检查原始KDB文件')

        if self.config['save_as_csv']:
            if (self.config['runD']): print(dt.datetime.now(), '检查: 是否存在原始CSV文件')
            dir = os.listdir(self.config['place_save']['csv'])
            for i in self.config['files_stored']:
                if i in dir: pass
                else: self.csv_initial_download_list.append(i)
            if len(self.csv_initial_download_list) > 0:
                if (self.config['runD']): print(dt.datetime.now(), '检查: 缺失以下CSV文件: 将下载最新的以下CSV文件: '+str(self.csv_initial_download_list))
                self.whether_download_initial_CSV = 1
            else:
                if (self.config['runD']): print(dt.datetime.now(), '检查: 不缺少原始CSV文件')
        else:
            if (self.config['runD']): print(dt.datetime.now(), '检查: 跳过检查原始CSV文件,开始检查文件是否最新')

In [ ]:
config_paths[i]#.replace('/','\')

In [ ]:
a = 'sdasd/sdsd'
a.replace('/','\')

In [ ]:
config_paths

In [10]:
ts_api = query_api.ts_api(config_download)

In [11]:
data_none = ts_api.get_hist_OHLCV('000017.SZ','D','E','None','20100701','20180801')
data_none

,ts_code,open,high,low,close,vol,amount,adj_factor
trade_date,,,,,,,,
2010-07-01,000017.SZ,5.07,5.15,5.02,5.05,10315.00,5239.7708,2.683
2010-07-02,000017.SZ,5.05,5.13,4.80,5.00,16850.44,8344.2486,2.683
2010-07-05,000017.SZ,4.92,5.02,4.89,4.99,10418.86,5174.9476,2.683
2010-07-06,000017.SZ,5.00,5.17,5.00,5.17,12488.94,6348.8501,2.683
2010-07-07,000017.SZ,5.15,5.16,5.04,5.12,14757.31,7532.8417,2.683
2010-07-08,000017.SZ,5.16,5.16,5.04,5.08,12743.83,6482.9666,2.683
2010-07-09,000017.SZ,5.09,5.19,5.04,5.17,14280.78,7339.0240,2.683
2010-07-12,000017.SZ,5.15,5.29,5.13,5.23,14970.62,7815.7017,2.683
2010-07-13,000017.SZ,5.21,5.25,4.99,5.04,17082.67,8640.1308,2.683


In [9]:
data_qfq = ts_api.get_hist_OHLCV('000017.SZ','D','E','qfq','20100701','20180801')
data_none = ts_api.get_hist_OHLCV('000017.SZ','D','E','None','20100701','20180801')

NameError: name 'ts_api' is not defined

In [ ]:
data_hfq = ts_api.get_hist_OHLCV('000017.SZ','D','E','hfq','20100701','20180801')

In [ ]:
pro = ts.pro_api(config_download['ts_api'])
pro.stock_basic(exchange_id='', list_status='L', fields='ts_code,symbol,list_date')

In [ ]:
data_qfq

In [ ]:
data_none

In [ ]:
data_none['close']*fc['adj_factor']

In [ ]:
fc['adj_factor']

In [ ]:
data_hfq

In [ ]:
pro = ts.pro_api(config_download['ts_api'])

In [ ]:
fc=pro.adj_factor(ts_code='000017.SZ', start_date='20100701',end_date='20180801')

In [ ]:
fc = fc.set_index(pd.to_datetime(fc['trade_date']))

In [ ]:
path = 'D:\Quant\Data\KDB\q\w32\save'         # 替换为你的路径
dir = os.listdir(path)                  # dir是目录下的全部文件
fopen = open('train_img_set/bg.txt', 'w') # 替换为你的路径
for d in dir:                        # d是每一个文件的文件名
    string = 'non-ball/' + d + '\n'    #拼接字符串并换行
    fopen.write(string)             # 写入文件中
fopen.close()

In [ ]:
D:\Quant\Data\KDB\q\w32\save

In [ ]:
import os
path = 'D:\Quant\Data\KDB\q\w32\save'         # 替换为你的路径
dir = os.listdir(path) 

In [ ]:
print('缺失以下KDB文件: 将下载以下文件: '+str(dir))

In [ ]:
a = 'trade_E_D_hfq_000929_20100101_20180930'

In [ ]:
if a in dir: print(a)